In [1]:
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_community.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
import shutil
import argparse
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
# from dataclasses import dataclass

DATA_PATH = "Data"

In [2]:
# Loading documents, using a local file from Alice In Wonderland Sample text

def load_documents():
    loader = DirectoryLoader(DATA_PATH, glob="AliceInWonderlandSample.md")
    documents = loader.load()
    return documents

documents = load_documents()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 500,
    length_function = len,
    add_start_index = True
)

chunks = text_splitter.split_documents(documents)

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


In [ ]:
CHROMA_PATH = "chroma"

import os
os.environ["OPENAI_API_KEY"] 

# Clear out the DB first
if os.path.exists(CHROMA_PATH):
    shutil.rmtree(CHROMA_PATH)

embedding_model = OpenAIEmbeddings()

# Create a new DB from the documents
db = Chroma.from_documents(
    chunks, OpenAIEmbeddings(), persist_directory = CHROMA_PATH
)

C:\Users\agila\AppData\Local\Temp\ipykernel_9844\3979188175.py:10: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings()


In [4]:
CHROMA_PATH = "chroma"

PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

# Define your query directly
query_text = "Who is Alice sitting beside?"

# Prepare the DB
embedding_function = OpenAIEmbeddings()
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

# Search the DB
results = db.similarity_search_with_relevance_scores(query_text, k=3)
if len(results) == 0 or results[0][1] < 0.7:
    print("Unable to find matching results.")
else:
    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)

    model = ChatOpenAI()
    response_text = model.predict(prompt)

    sources = [doc.metadata.get("source", None) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)

C:\Users\agila\AppData\Local\Temp\ipykernel_9844\2800758302.py:18: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)
C:\Users\agila\AppData\Local\Temp\ipykernel_9844\2800758302.py:30: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response_text = model.predict(prompt)


Response: Alice is sitting beside her brother on the bank.
Sources: ['Data\\AliceInWonderlandSample.md']
